# this should be in backend
testing here for convenience

In [1]:
import torch
import pandas as pd
import json
from openai import OpenAI
from sentence_transformers import SentenceTransformer, util

from svs import SVS
from queryOpenAI import run_engine

/Users/isaac/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# preloading slow things
model = SentenceTransformer('distilbert-multilingual-nli-stsb-quora-ranking')
f = open('../models/mimic-iii/qqdict.json')
qqdict = json.load(f)
corpus_embedding = torch.tensor(pd.read_csv("../models/mimic-iii/corpus.csv").values).to(torch.float32)

client = OpenAI(api_key=os.getenv("OAI_KEY"))

/Users/isaac/Library/Python/3.9/lib/python/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [3]:
question = "What antibiotic was prescribed to patient 1125?"

In [4]:
def backend(question):
    qqlist = [qqdict[str(i)] for i in SVS(question, k=15, corpus=corpus_embedding, model=model)]

    relevant_qtext = "# Q: \t" + "\n# Q: \t".join(["\n# A: \t".join(rel) for rel in qqlist])
    test_prompt = open('../models/mimic-iii/codex_apidoc.txt').read()

    result = run_engine(question, test_prompt, relevant_qtext)

    print(result)
    print()
    return result
    
backend(question)

# no LIMIT allowed 
#
# hadm_id: hospital admission id
# dose_val_rx: dosage size
# dose_unit_rx: unit of dose
# valueom: value units over minutes 
#
### MySQL SQL tables, with their properties:
#
# patients(row_id, subject_id, gender, dob, dod)
# admissions(row_id, subject_id, hadm_id, admittime, dischtime, admission_type, admission_location, discharge_location, insurance, language, marital_status, ethnicity, age)
# d_icd_diagnoses(row_id, icd9_code, short_title, long_title)
# d_icd_procedures(row_id, icd9_code, short_title, long_title)
# d_labitems(row_id, itemid, label)
# d_items(row_id, itemid, label, linksto)
# diagnoses_icd(row_id, subject_id, hadm_id, icd9_code, charttime)
# procedures_icd(row_id, subject_id, hadm_id, icd9_code, charttime)
# labevents(row_id, subject_id, hadm_id, itemid, charttime, valuenum, valueuom)
# prescriptions(row_id, subject_id, hadm_id, startdate, enddate, drug, dose_val_rx, dose_unit_rx, route)
# cost(row_id, subject_id, hadm_id, event_type, event_id, 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



SELECT DRUG 
FROM PRESCRIPTIONS 
WHERE SUBJECT_ID = 1125 
AND DRUG LIKE '%ANTIBIOTIC%';


"SELECT DRUG \nFROM PRESCRIPTIONS \nWHERE SUBJECT_ID = 1125 \nAND DRUG LIKE '%ANTIBIOTIC%';"

In [20]:
test_prompt = open('../models/mimic-iii/codex_apidoc.txt').read()

In [5]:
print(question)
print()
print("SELECT COUNT(*)>0 \nFROM PRESCRIPTIONS \nWHERE PRESCRIPTIONS.HADM_ID IN \n( \n    SELECT ICUSTAYS.HADM_ID \n    FROM ICUSTAYS \n    WHERE ICUSTAYS.SUBJECT_ID = 22973 \n    ORDER BY ICUSTAYS.INTIME DESC \n    LIMIT 1 \n) \nAND PRESCRIPTIONS.DRUG = 'INSULIN';")

Was patient 22973 prescribed insulin the last time they were at the ICU?

SELECT COUNT(*)>0 
FROM PRESCRIPTIONS 
WHERE PRESCRIPTIONS.HADM_ID IN 
( 
    SELECT ICUSTAYS.HADM_ID 
    FROM ICUSTAYS 
    WHERE ICUSTAYS.SUBJECT_ID = 22973 
    ORDER BY ICUSTAYS.INTIME DESC 
    LIMIT 1 
) 
AND PRESCRIPTIONS.DRUG = 'INSULIN';
